In [0]:
dbutils.fs.ls('mnt/gold/')


In [0]:
input_path ='/mnt/silver/SalesLT/Address/'


In [0]:
#spark.conf.set("spark.databricks.delta.formatCheck.enabled", "false")
df = spark.read.format('delta').load(input_path)

In [0]:
display(df)

In [0]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import col, regexp_replace

column_names = df.columns

for old_col_name in column_names:
    new_col_name = "".join(["_"+ char if char.isupper() and not old_col_name[i-1].isupper() else char for i, char in enumerate(old_col_name)]).lstrip("_")

    # Change the column name using withColumnRenamed and regexp_replace
    df = df.withColumnRenamed(old_col_name, new_col_name)



In [0]:
display(df)

## Doing transformation for all tables

In [0]:
table_name=[]

for i in dbutils.fs.ls('mnt/silver/SalesLT/'):
	table_name.append(i.name.split('/')[0])

In [0]:
for name in table_name:
    path='/mnt/silver/SalesLT/' + name
    print(path)
    df =  spark.read.format('delta').load(path)

    #Get the list of column names
    column_names = df.columns

    for old_col_name in column_names:
        # Convert column name for ColumnName to Column_Name format
        new_col_name = "".join(["_"+ char if char.isupper() and not old_col_name[i-1].isupper() else char for i, char in enumerate(old_col_name)]).lstrip("_")

        df = df.withColumnRenamed(old_col_name, new_col_name)
    
    output_path = '/mnt/gold/SalesLT/' + name + '/'
    df.write.format('delta').mode("overwrite").save(output_path)

